# MolMap ensemble (dual-path approach)

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]='0'

import tensorflow as tf

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

### Load data

In [2]:
'Set the file name of your data'
data_name = "CYP450.csv.gz"

In [3]:
import numpy as np
import pandas as pd

data = pd.read_csv("../../../data/" + data_name, compression='gzip')

Set up (adapt according to your data)

In [4]:
# keep desired columns
data = data[['smiles', 'label_2c9']]

# drop molecules with NaN activity
data = data.dropna(subset = ["label_2c9"])

# set SMILES
smi = data['smiles'].tolist()

# set Y
Y = pd.get_dummies(data['label_2c9']).values

# number of active and inactive molecules
print("Inactive (0):", Y[:,1].tolist().count(0))
print("Active (1):", Y[:,1].tolist().count(1))

Inactive (0): 7429
Active (1): 2621


### MolMap Ensemble

In [5]:
'Set size of class 0 (minority class) for each model'
size_0 = 500

partitions = int(np.floor(Y[:,1].tolist().count(0)/size_0))
print("Total number of models:", partitions)

Total number of models: 14


In [6]:
'Get classes 0 and 1 in two different variables'
class_0 = data[data["label_2c9"] == 0.0]
class_1 = data[data["label_2c9"] == 1.0]

# shuffle class 0 before dividing it into parts
class_0 = class_0.sample(frac=1).reset_index(drop=True)

In [7]:
from molmap import MolMap
from molmap import feature
from molmap.model import MultiClassEstimator
import sys
sys.path.append("../../../src")
from utils import Rdsplit

aucs = []
# for each model:
for i in range (0, partitions):
    
    # subset of data for model i
    subdata = pd.concat([class_0[i*size_0:(i+1)*size_0], class_1])
    
    # get smiles
    smi = subdata['smiles']
    
    # set Y
    Y = pd.get_dummies(subdata['label_2c9']).values
    
    # compute MolDs
    mp1 = MolMap(ftype='descriptor', metric='cosine',)
    mp1.fit(verbose=0, method='umap', min_dist=0.1, n_neighbors=15)
    
    # compute FFs
    bitsinfo = feature.fingerprint.Extraction().bitsinfo
    flist = bitsinfo[bitsinfo.Subtypes.isin(['PubChemFP', 'MACCSFP', 'PharmacoErGFP'])].IDs.tolist()
    mp2 = MolMap(ftype = 'fingerprint', fmap_type = 'scatter', flist = flist) 
    mp2.fit(method = 'umap',  min_dist = 0.1, n_neighbors = 15, verbose = 0)
    
    # get Fmaps
    X1 = mp1.batch_transform(smi)
    X2 = mp2.batch_transform(smi)
    
    # split train, val, test
    train_idx, valid_idx, test_idx = Rdsplit(subdata, random_state = 888)
    trainX = (X1[train_idx], X2[train_idx])
    validX = (X1[valid_idx], X2[valid_idx])
    testX = (X1[test_idx], X2[test_idx])
    trainY = Y[train_idx]
    validY = Y[valid_idx]
    testY = Y[test_idx]
    
    # model 
    clf = MultiClassEstimator(n_outputs=trainY.shape[1], 
                              fmap_shape1 = X1.shape[1:],
                              fmap_shape2 = X2.shape[1:],
                              metric='ROC', 
                              dense_layers = [128, 64],  gpuid = 0, epochs = 100,
                              ) 
    # fit model
    clf.fit(trainX, trainY, validX, validY)
    
    # score 
    auc = clf.score(testX, testY) 
    aucs.append(auc)

2022-06-29 11:55:59,775 - INFO - [bidd-molmap] - Applying grid feature map(assignment), this may take several minutes(1~30 min)
2022-06-29 11:56:02,605 - INFO - [bidd-molmap] - Finished
2022-06-29 11:56:09,629 - INFO - [bidd-molmap] - Applying naive scatter feature map...
2022-06-29 11:56:09,648 - INFO - [bidd-molmap] - Finished


100%|##########| 3121/3121 [04:14<00:00, 12.28it/s]


2497 312 312
MultiClassEstimator(epochs=100, fmap_shape1=(37, 37, 13),
                    fmap_shape2=(72, 72, 3), gpuid='0', n_outputs=2)
epoch: 0001, loss: 0.4767 - val_loss: 0.4741; auc: 0.6944 - val_auc: 0.6896                                                                                                    
epoch: 0002, loss: 0.4208 - val_loss: 0.4471; auc: 0.7616 - val_auc: 0.7355                                                                                                    
epoch: 0003, loss: 0.4083 - val_loss: 0.4359; auc: 0.7842 - val_auc: 0.7542                                                                                                    
epoch: 0004, loss: 0.3916 - val_loss: 0.4289; auc: 0.7930 - val_auc: 0.7622                                                                                                    
epoch: 0005, loss: 0.3789 - val_loss: 0.4101; auc: 0.8110 - val_auc: 0.7782                                                                                 

100%|##########| 3121/3121 [02:24<00:00, 21.53it/s]


2497 312 312
MultiClassEstimator(epochs=100, fmap_shape1=(37, 37, 13),
                    fmap_shape2=(72, 72, 3), gpuid='0', n_outputs=2)
epoch: 0001, loss: 0.4764 - val_loss: 0.4642; auc: 0.7003 - val_auc: 0.7660                                                                                                    
epoch: 0002, loss: 0.4200 - val_loss: 0.4364; auc: 0.7637 - val_auc: 0.7982                                                                                                    
epoch: 0003, loss: 0.4066 - val_loss: 0.4199; auc: 0.7910 - val_auc: 0.8139                                                                                                    
epoch: 0004, loss: 0.3911 - val_loss: 0.4025; auc: 0.8047 - val_auc: 0.8220                                                                                                    
epoch: 0005, loss: 0.3767 - val_loss: 0.3834; auc: 0.8165 - val_auc: 0.8267                                                                                 

...

2497 312 312
MultiClassEstimator(epochs=100, fmap_shape1=(37, 37, 13),
                    fmap_shape2=(72, 72, 3), gpuid='0', n_outputs=2)
epoch: 0001, loss: 0.4767 - val_loss: 0.4691; auc: 0.7029 - val_auc: 0.7304                                                                                                    
epoch: 0002, loss: 0.4200 - val_loss: 0.4413; auc: 0.7637 - val_auc: 0.7861                                                                                                    
epoch: 0003, loss: 0.4079 - val_loss: 0.4259; auc: 0.7875 - val_auc: 0.8084                                                                                                    
epoch: 0004, loss: 0.3930 - val_loss: 0.4105; auc: 0.8011 - val_auc: 0.8203                                                                                                    
epoch: 0005, loss: 0.3791 - val_loss: 0.3936; auc: 0.8156 - val_auc: 0.8322                                                                                 

100%|##########| 3121/3121 [01:37<00:00, 32.00it/s]


2497 312 312
MultiClassEstimator(epochs=100, fmap_shape1=(37, 37, 13),
                    fmap_shape2=(72, 72, 3), gpuid='0', n_outputs=2)
epoch: 0001, loss: 0.4773 - val_loss: 0.4757; auc: 0.6837 - val_auc: 0.6519                                                                                                    
epoch: 0002, loss: 0.4211 - val_loss: 0.4492; auc: 0.7553 - val_auc: 0.7059                                                                                                    
epoch: 0003, loss: 0.4084 - val_loss: 0.4385; auc: 0.7845 - val_auc: 0.7317                                                                                                    
epoch: 0004, loss: 0.3947 - val_loss: 0.4311; auc: 0.7974 - val_auc: 0.7411                                                                                                    
epoch: 0005, loss: 0.3802 - val_loss: 0.4187; auc: 0.8105 - val_auc: 0.7515                                                                                 

100%|##########| 3121/3121 [01:39<00:00, 31.52it/s]


2497 312 312
MultiClassEstimator(epochs=100, fmap_shape1=(37, 37, 13),
                    fmap_shape2=(72, 72, 3), gpuid='0', n_outputs=2)
epoch: 0001, loss: 0.4768 - val_loss: 0.4704; auc: 0.6912 - val_auc: 0.7158                                                                                                    
epoch: 0002, loss: 0.4218 - val_loss: 0.4446; auc: 0.7528 - val_auc: 0.7588                                                                                                    
epoch: 0003, loss: 0.4109 - val_loss: 0.4322; auc: 0.7800 - val_auc: 0.7776                                                                                                    
epoch: 0004, loss: 0.3974 - val_loss: 0.4250; auc: 0.7918 - val_auc: 0.7843                                                                                                    
epoch: 0005, loss: 0.3858 - val_loss: 0.4058; auc: 0.8144 - val_auc: 0.7987                                                                                 

In [8]:
print("Total number of models trained:", partitions)
print("Mean of AUCs:", round(np.mean(aucs), 3))
print("Variance of AUCs:", round(np.var(aucs), 3))

Total number of models trained: 14
Mean of accuracies: 0.894
Variance of accuracies: 0.0
